Loading all the libraris that i will use.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t

Loading the QVI_data.csv to a variable data

In [20]:

data = pd.read_csv('task_two_data.csv').drop('Unnamed: 0', axis = 1).reset_index(drop=True)
data.index +=1
data = pd.DataFrame(data)
data.head()

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,PRICE_PER_UNIT
1,1000,YOUNG SINGLES/COUPLES,Premium,2018-10-17,1.0,1.0,5.0,Natural Chip Compny SeaSalt175g,2.0,6.0,175.0,Natural,3.0
2,1002,YOUNG SINGLES/COUPLES,Mainstream,2018-09-16,1.0,2.0,58.0,Red Rock Deli Chikn&Garlic Aioli 150g,1.0,2.7,150.0,Red,2.7
3,1003,YOUNG FAMILIES,Budget,2019-03-08,1.0,4.0,106.0,Natural ChipCo Hony Soy Chckn175g,1.0,3.0,175.0,Natural,3.0
4,1004,OLDER SINGLES/COUPLES,Mainstream,2018-11-02,1.0,5.0,96.0,WW Original Stacked Chips 160g,1.0,1.9,160.0,WW,1.9
5,1005,MIDAGE SINGLES/COUPLES,Mainstream,2018-12-28,1.0,6.0,86.0,Cheetos Puffs 165g,1.0,2.8,165.0,Cheetos,2.8


lets check the store with trials 77, 86 and 88

In [21]:
trial_stores = data[data['STORE_NBR'].isin([77.0, 86.0,88.0])]
trial_store

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,PRICE_PER_UNIT
66697,77000,MIDAGE SINGLES/COUPLES,Budget,2019-03-28,77.0,74911.0,18.0,Cheetos Chs & Bacon Balls 190g,1.0,3.3,190.0,Cheetos,3.3
66698,77000,MIDAGE SINGLES/COUPLES,Budget,2019-04-13,77.0,74912.0,69.0,Smiths Chip Thinly S/Cream&Onion 175g,1.0,3.0,175.0,Smiths,3.0
66699,77000,MIDAGE SINGLES/COUPLES,Budget,2018-09-26,77.0,74910.0,36.0,Kettle Chilli 175g,2.0,10.8,175.0,Kettle,5.4
66700,77001,YOUNG FAMILIES,Mainstream,2019-02-27,77.0,74913.0,7.0,Smiths Crinkle Original 330g,2.0,11.4,330.0,Smiths,5.7
66701,77001,YOUNG FAMILIES,Mainstream,2019-01-21,77.0,74914.0,9.0,Kettle Tortilla ChpsBtroot&Ricotta 150g,2.0,9.2,150.0,Kettle,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240672,2370651,MIDAGE SINGLES/COUPLES,Mainstream,2018-08-03,88.0,240350.0,4.0,Dorito Corn Chp Supreme 380g,2.0,13.0,380.0,Dorito,6.5
240673,2370701,YOUNG FAMILIES,Mainstream,2018-12-08,88.0,240378.0,24.0,Grain Waves Sweet Chilli 210g,2.0,7.2,210.0,Grain,3.6
240674,2370751,YOUNG FAMILIES,Premium,2018-10-01,88.0,240394.0,60.0,Kettle Tortilla ChpsFeta&Garlic 150g,2.0,9.2,150.0,Kettle,4.6
240675,2370961,OLDER FAMILIES,Budget,2018-10-24,88.0,240480.0,70.0,Tyrrells Crisps Lightly Salted 165g,2.0,8.4,165.0,Tyrrells,4.2


#### Calculate Total Sales Revenue, Total Number of Customers, and Average Transactions per Customer

In [36]:
# Calculate Total Sales Revenue
total_sales_revenue = data['TOT_SALES'].sum()

# Calculate Total Number of Customers
total_customers = data['LYLTY_CARD_NBR'].nunique()

# Calculate Average Transactions per Customer
average_transactions_per_customer = len(data) / total_customers

# Display the results
print(f'Total Sales Revenue: ${total_sales_revenue:.2f}')
print(f'Total Number of Customers: {total_customers}')
print(f'Average Transactions per Customer: {average_transactions_per_customer:.2f}')

Total Sales Revenue: $1767825.90
Total Number of Customers: 70827
Average Transactions per Customer: 3.40


### normalizing metrics
Normalization is the process of scaling numeric data to a standard range, typically between 0 and 1 or -1 and 1. Normalizing metrics is important because it ensures that all the features (metrics) are on the same scale, which can be crucial for some machine learning algorithms and visualizations.

In this case, we want to normalize the following metrics:

Total Sales Revenue

Total Number of Customers

Average Transactions per Customer

In [27]:
def find_max_frequency(df, group_column):
    # Group the DataFrame by the specified column
    grouped = df.groupby(group_column)

    # Count the frequency of each value in the grouped column
    frequency_counts = grouped.size()

    # Find the maximum frequency count
    max_frequency = frequency_counts.max()

    return max_frequency
find_max_frequency(data, 'STORE_NBR')

1894

In [31]:
# Define the maximum values for each metric
max_total_sales = max(data['TOT_SALES'])
max_total_customers = max(data['LYLTY_CARD_NBR'])
max_transactions = find_max_frequency(data, 'STORE_NBR')

# Normalize the metrics by dividing by their maximum values
normalized_total_sales = 1767825.90 / max_total_sales
normalized_total_customers = 70827 / max_total_customers
normalized_avg_transactions = 3.40 / max_transactions

# Create a dictionary to store the normalized metrics
normalized_metrics = {
    'Normalized Total Sales Revenue': normalized_total_sales,
    'Normalized Total Number of Customers': normalized_total_customers,
    'Normalized Average Transactions per Customer': normalized_avg_transactions
}

# Display the normalized metrics
print("Normalized Metrics:")
for metric, value in normalized_metrics.items():
    print(f"{metric}: {value:.5f}")

Normalized Metrics:
Normalized Total Sales Revenue: 59926.30169
Normalized Total Number of Customers: 0.02984
Normalized Average Transactions per Customer: 0.00180
